In [2]:
import pandas as pd
from textblob import TextBlob
from deep_translator import GoogleTranslator
import langid
import math

data = pd.read_csv("./kinton_reviews.csv")
train = pd.read_csv("./data/train.csv")

In [3]:
reviews = data[["review_rating", "review_text", "review_questions_Meal type"]].dropna(subset=["review_text"])

def getPolarity(text):
    text = str(text)
    try:
        lang = langid.classify(text)[0]
    except Exception:
        lang = 'en'
    
    if lang != 'en':
        try:
            text = GoogleTranslator(source='auto', target='en').translate(text=text)
        except Exception:
            pass

    return TextBlob(text).sentiment.polarity

In [5]:
print("Calculating sentiment polarity...")
reviews["polarity"] = reviews["review_text"].apply(getPolarity)

reviews["sentiment_model"] = reviews["polarity"].apply(
    lambda x: 'positive' if x > 0.15 else ('negative' if x < -0.15 else 'neutral')
)

reviews["sentiment_true"] = reviews["review_rating"].apply(
    lambda x: 'positive' if x > 3 else ('negative' if x < 3 else 'neutral')
)

reviews["accuracy"] = reviews["sentiment_model"] == reviews["sentiment_true"]

reviews.to_csv("kinton_sentiment_results.csv", index=False)
print("'kinton_sentiment_results.csv' has been saved successfully!")

print("\nSummary:")
print(reviews["sentiment_model"].value_counts())
print("\nOverall accuracy:", round(reviews["accuracy"].mean()*100, 2), "%")

Calculating sentiment polarity...
'kinton_sentiment_results.csv' has been saved successfully!

Summary:
sentiment_model
positive    358
neutral      52
negative      5
Name: count, dtype: int64

Overall accuracy: 86.51 %
